# 🚀 Messung der OCR-Qualität in Python

<div class="alert alert-block alert-info"> <b> 🔔 Feinlernziel(e) dieses Kapitels</b></br>
   Sie können die Schritte zur Qualitätsmessung eines OCR-Outputs aufzählen und die Qualitätsmaße interpretieren.</div>

In diesem Notebook bewerten wir die OCR-Qualität. 

## Installationen und Importe

In [ ]:
# 🚀 Install libraries  
!sudo apt install tesseract-ocr
!sudo apt install tesseract-ocr-frk
!sudo apt install poppler-utils
!pip install pytesseract pillow Levenshtein

In [ ]:
import pytesseract
from PIL import Image
from pathlib import Path

## for interactivity in jupyter books
from bokeh.io import output_notebook, show
from bokeh.layouts import column
from bokeh.models import CustomJS, TextInput

## Durchführen von OCR

Wir werden denselben Beispielartikel zur Bewertung verwenden:
<!-- In this notebook we evaluate OCR quality. We'll use the same sample article for evaluation: -->

![grippe.png](grippe.png)

In [ ]:
# 🚀 get the image to process in Colab
!wget https://raw.githubusercontent.com/dh-network/quadriga/refs/heads/main/data-input/grippe.jpeg

Zuerst führen wir die OCR durch: <!-- Let us OCR it first: -->

In [ ]:
ocr_output = pytesseract.image_to_string(Image.open('grippe.jpeg'), 
                                         lang='frk') # Wir verwenden das Fraktur-OCR-Modell

In [ ]:
print(ocr_output)

### Manuell die 'Ground Truth' zur Bewertung erstellen <!-- ### Manually create  the 'ground truth' to evaluate against -->

In [ ]:
output_notebook()

capture_ground_truth = TextInput(value="""Die Grippe wütet weiter. Zunahme der schweren Fälle in Berlin. Die Zahl der Grippefälle ist in den letzten Tagen auch in Groß-Berlin noch erheblich gestiegen. Die Warenhäuser und sonstigen großen Geschäfte, die Kriegs- und die privaten Betriebe klagen, daß übermäßig viele Angestellte sich haben krank melden müssen und auch bei der Post und bei der Straßenbahn ist der Prozentsatz der Grippekranken deutlich gestiegen.""", 
                                 title="Please insert corrected string: ") 

ground_truth = capture_ground_truth.value.strip()

rewrite_var_after_input = CustomJS(args=dict(text_input=capture_ground_truth), code="""
    var word = text_input.value.trim();
    console.log('Input value:', word);
    function sendToPython(){
    var kernel = IPython.notebook.kernel;
    kernel.execute("ground_truth = '" + word + "'");
    }
    sendToPython();
""")



capture_ground_truth.js_on_change('value', rewrite_var_after_input)

In [ ]:
# Layout and display
layout = column(capture_ground_truth)

show(layout)

In [ ]:
print(ground_truth)

## Berechnung von Precision, Recall und F1-Measure <!-- ### Calculate precision, recall, and F1-measure -->

In [ ]:
# 🚀 create folder and get the auxiliary python script to run in Colab
aux_dir = Path("auxiliary")
if not aux_dir.exists():
    aux_dir.mkdir(parents=True)

In [ ]:
# 🚀 create folder and get the auxiliary python script to run in Colab
!wget https://raw.githubusercontent.com/dh-network/quadriga/refs/heads/main/data-input/auxiliary/measure_ocr_quality.py
!mv measure_ocr_quality.py auxiliary/

In [ ]:
from auxiliary.measure_ocr_quality import measure_ocr_quality

In [ ]:
precision, recall, f_score = measure_ocr_quality(ocr_output, ground_truth)

In [ ]:
print(f'Precision: {round(precision, 4)}\nRecall: {round(recall, 4)}\nF1-score: {round(f_score, 4)}')